In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./CICFlowMeter_out.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540241 entries, 0 to 3540240
Data columns (total 84 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Flow ID                     object 
 1   Src IP                      object 
 2   Src Port                    int64  
 3   Dst IP                      object 
 4   Dst Port                    int64  
 5   Protocol                    int64  
 6   Timestamp                   object 
 7   Flow Duration               int64  
 8   Total Fwd Packet            int64  
 9   Total Bwd packets           int64  
 10  Total Length of Fwd Packet  float64
 11  Total Length of Bwd Packet  float64
 12  Fwd Packet Length Max       float64
 13  Fwd Packet Length Min       float64
 14  Fwd Packet Length Mean      float64
 15  Fwd Packet Length Std       float64
 16  Bwd Packet Length Max       float64
 17  Bwd Packet Length Min       float64
 18  Bwd Packet Length Mean      float64
 19  Bwd Packet Length Std

In [4]:
df.to_parquet(f"./CIC-UNSW-NB15.parquet")

In [5]:
df1 = pd.read_parquet('./CIC-UNSW-NB15.parquet', engine='pyarrow')

In [6]:
df1 = df1.drop(df1.columns[[0, 1, 2, 3, 4, 6]], axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540241 entries, 0 to 3540240
Data columns (total 78 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Protocol                    int64  
 1   Flow Duration               int64  
 2   Total Fwd Packet            int64  
 3   Total Bwd packets           int64  
 4   Total Length of Fwd Packet  float64
 5   Total Length of Bwd Packet  float64
 6   Fwd Packet Length Max       float64
 7   Fwd Packet Length Min       float64
 8   Fwd Packet Length Mean      float64
 9   Fwd Packet Length Std       float64
 10  Bwd Packet Length Max       float64
 11  Bwd Packet Length Min       float64
 12  Bwd Packet Length Mean      float64
 13  Bwd Packet Length Std       float64
 14  Flow Bytes/s                float64
 15  Flow Packets/s              float64
 16  Flow IAT Mean               float64
 17  Flow IAT Std                float64
 18  Flow IAT Max                float64
 19  Flow IAT Min         

In [7]:
df1['Label'].value_counts()

Label
Benign            3450658
Exploits            30951
Fuzzers             29613
Reconnaissance      16735
Generic              4632
DoS                  4467
Shellcode            2102
Backdoor              452
Analysis              385
Worms                 246
Name: count, dtype: int64

In [8]:
df_BENIGN = df1.loc[df1['Label'] == 'Benign']
df_BENIGN['Label'].value_counts()

Label
Benign    3450658
Name: count, dtype: int64

In [9]:
df_Scan1 = df1.loc[df1['Label'] == 'Fuzzers']
df_Scan1['Label'].value_counts()

Label
Fuzzers    29613
Name: count, dtype: int64

In [10]:
df_Scan2 = df1.loc[df1['Label'] == 'Reconnaissance']
df_Scan2['Label'].value_counts()

Label
Reconnaissance    16735
Name: count, dtype: int64

In [11]:
df_Scan3 = df1.loc[df1['Label'] == 'Analysis']
df_Scan3['Label'].value_counts()

Label
Analysis    385
Name: count, dtype: int64

In [12]:
df_Worm1 = df1.loc[df1['Label'] == 'Exploits']
df_Worm1['Label'].value_counts()

Label
Exploits    30951
Name: count, dtype: int64

In [13]:
df_Worm2 = df1.loc[df1['Label'] == 'Backdoor']
df_Worm2['Label'].value_counts()

Label
Backdoor    452
Name: count, dtype: int64

In [14]:
df_Worm3 = df1.loc[df1['Label'] == 'Shellcode']
df_Worm3['Label'].value_counts()

Label
Shellcode    2102
Name: count, dtype: int64

In [15]:
df_Worm4 = df1.loc[df1['Label'] == 'Worms']
df_Worm4['Label'].value_counts()

Label
Worms    246
Name: count, dtype: int64

In [16]:
df_coat = pd.concat([df_BENIGN, df_Scan1, df_Scan2, df_Scan3, df_Worm1, df_Worm2, df_Worm3, df_Worm4])

In [17]:
df_coat['Label'].value_counts()

Label
Benign            3450658
Exploits            30951
Fuzzers             29613
Reconnaissance      16735
Shellcode            2102
Backdoor              452
Analysis              385
Worms                 246
Name: count, dtype: int64

In [18]:
df_coat['Label'] = df_coat['Label'].str.replace('Fuzzers', 'Scan')

In [19]:
df_coat['Label'] = df_coat['Label'].str.replace('Reconnaissance', 'Scan')

In [20]:
df_coat['Label'] = df_coat['Label'].str.replace('Analysis', 'Scan')

In [21]:
df_coat['Label'] = df_coat['Label'].str.replace('Exploits', 'Worm')

In [22]:
df_coat['Label'] = df_coat['Label'].str.replace('Backdoor', 'Worm')

In [23]:
df_coat['Label'] = df_coat['Label'].str.replace('Shellcode', 'Worm')

In [24]:
df_coat['Label'] = df_coat['Label'].str.replace('Worms', 'Worm')

In [25]:
df_coat['Label'].value_counts()

Label
Benign    3450658
Scan        46733
Worm        33751
Name: count, dtype: int64

In [26]:
attack = df_coat.loc[:, "Label"].map(lambda lbl: "0" if lbl == "Benign" else "1")
attack.name = "Attack"
df_coat.loc[:, attack.name] = attack

df_coat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531142 entries, 26 to 3508839
Data columns (total 79 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Protocol                    int64  
 1   Flow Duration               int64  
 2   Total Fwd Packet            int64  
 3   Total Bwd packets           int64  
 4   Total Length of Fwd Packet  float64
 5   Total Length of Bwd Packet  float64
 6   Fwd Packet Length Max       float64
 7   Fwd Packet Length Min       float64
 8   Fwd Packet Length Mean      float64
 9   Fwd Packet Length Std       float64
 10  Bwd Packet Length Max       float64
 11  Bwd Packet Length Min       float64
 12  Bwd Packet Length Mean      float64
 13  Bwd Packet Length Std       float64
 14  Flow Bytes/s                float64
 15  Flow Packets/s              float64
 16  Flow IAT Mean               float64
 17  Flow IAT Std                float64
 18  Flow IAT Max                float64
 19  Flow IAT Min             

In [27]:
df_coat['Attack'] = df_coat['Attack'].astype(int)

In [28]:
df_coat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531142 entries, 26 to 3508839
Data columns (total 79 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Protocol                    int64  
 1   Flow Duration               int64  
 2   Total Fwd Packet            int64  
 3   Total Bwd packets           int64  
 4   Total Length of Fwd Packet  float64
 5   Total Length of Bwd Packet  float64
 6   Fwd Packet Length Max       float64
 7   Fwd Packet Length Min       float64
 8   Fwd Packet Length Mean      float64
 9   Fwd Packet Length Std       float64
 10  Bwd Packet Length Max       float64
 11  Bwd Packet Length Min       float64
 12  Bwd Packet Length Mean      float64
 13  Bwd Packet Length Std       float64
 14  Flow Bytes/s                float64
 15  Flow Packets/s              float64
 16  Flow IAT Mean               float64
 17  Flow IAT Std                float64
 18  Flow IAT Max                float64
 19  Flow IAT Min             

In [29]:
df_coat.to_parquet(f"./CoAt_CIC-UNSW-NB15.parquet")

In [30]:
df_coat_origin = pd.read_parquet('./CoAt_CIC-UNSW-NB15.parquet', engine='pyarrow')
df_coat_feed = pd.read_parquet('/home/administrator/dataset/CoAt/feeding-dataset/Feed_DDoS_CIC-UNSW-NB15.parquet', engine='pyarrow')

In [31]:
df_coat_origin.rename(columns = {'Total Fwd Packet':'Total Fwd Packets', 
                                   'Total Bwd packets':'Total Backward Packets',
                                   'Total Length of Fwd Packet':'Fwd Packets Length Total',
                                   'Total Length of Bwd Packet':'Bwd Packets Length Total',
                                   'CWR Flag Count':'CWE Flag Count',
                                   'Average Packet Size':'Avg Packet Size',
                                   'Fwd Segment Size Avg':'Avg Fwd Segment Size',
                                   'Bwd Segment Size Avg':'Avg Bwd Segment Size',
                                   'Fwd Bytes/Bulk Avg':'Fwd Avg Bytes/Bulk',
                                   'Fwd Packet/Bulk Avg':'Fwd Avg Packets/Bulk',
                                   'Fwd Bulk Rate Avg':'Fwd Avg Bulk Rate',
                                   'Bwd Bytes/Bulk Avg':'Bwd Avg Bytes/Bulk',
                                   'Bwd Packet/Bulk Avg':'Bwd Avg Packets/Bulk',
                                   'Bwd Bulk Rate Avg':'Bwd Avg Bulk Rate',
                                   'Fwd Act Data Pkts':'Fwd Act Data Packets',
                                   'Bwd Init Win Bytes':'Init Bwd Win Bytes',
                                   'FWD Init Win Bytes':'Init Fwd Win Bytes'}, inplace = True)

In [32]:
df_coat_cmb = pd.concat([df_coat_origin, df_coat_feed])

In [11]:
df_coat_cmb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3630918 entries, 26 to 362173
Data columns (total 79 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Protocol                  int64  
 1   Flow Duration             int64  
 2   Total Fwd Packets         int64  
 3   Total Backward Packets    int64  
 4   Fwd Packets Length Total  float64
 5   Bwd Packets Length Total  float64
 6   Fwd Packet Length Max     float64
 7   Fwd Packet Length Min     float64
 8   Fwd Packet Length Mean    float64
 9   Fwd Packet Length Std     float64
 10  Bwd Packet Length Max     float64
 11  Bwd Packet Length Min     float64
 12  Bwd Packet Length Mean    float64
 13  Bwd Packet Length Std     float64
 14  Flow Bytes/s              float64
 15  Flow Packets/s            float64
 16  Flow IAT Mean             float64
 17  Flow IAT Std              float64
 18  Flow IAT Max              float64
 19  Flow IAT Min              float64
 20  Fwd IAT Total             flo

In [12]:
df_coat_cmb['Label'].value_counts()

Label
Benign    3450658
DDoS        95144
Scan        46348
Worm        38768
Name: count, dtype: int64

In [13]:
df_coat_cmb['Attack'].value_counts()

Attack
0    3450658
1     180260
Name: count, dtype: int64

In [14]:
df_coat_cmb.to_parquet(f"./CoAt_CIC-UNSW-NB15_Feeded.parquet")

In [18]:
df_origin = pd.read_parquet('./CoAt_CIC-UNSW-NB15_Feeded.parquet', engine='pyarrow')
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3630918 entries, 26 to 362173
Data columns (total 79 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Protocol                  int64  
 1   Flow Duration             int64  
 2   Total Fwd Packets         int64  
 3   Total Backward Packets    int64  
 4   Fwd Packets Length Total  float64
 5   Bwd Packets Length Total  float64
 6   Fwd Packet Length Max     float64
 7   Fwd Packet Length Min     float64
 8   Fwd Packet Length Mean    float64
 9   Fwd Packet Length Std     float64
 10  Bwd Packet Length Max     float64
 11  Bwd Packet Length Min     float64
 12  Bwd Packet Length Mean    float64
 13  Bwd Packet Length Std     float64
 14  Flow Bytes/s              float64
 15  Flow Packets/s            float64
 16  Flow IAT Mean             float64
 17  Flow IAT Std              float64
 18  Flow IAT Max              float64
 19  Flow IAT Min              float64
 20  Fwd IAT Total             flo

In [19]:
df_origin['Protocol'] = df_origin['Protocol'].astype('int8')
df_origin['Total Fwd Packets'] = df_origin['Total Fwd Packets'].astype('int32')
df_origin['Total Backward Packets'] = df_origin['Total Backward Packets'].astype('int32')
df_origin['Fwd Packet Length Min'] = df_origin['Fwd Packet Length Min'].astype('float32')
df_origin['Fwd Packet Length Mean'] = df_origin['Fwd Packet Length Mean'].astype('float32')
df_origin['Fwd Packet Length Std'] = df_origin['Fwd Packet Length Std'].astype('float32')
df_origin['Bwd Packet Length Min'] = df_origin['Bwd Packet Length Min'].astype('float32')
df_origin['Bwd Packet Length Mean'] = df_origin['Bwd Packet Length Mean'].astype('float32')
df_origin['Bwd Packet Length Std'] = df_origin['Bwd Packet Length Std'].astype('float32')
df_origin['Flow IAT Mean'] = df_origin['Flow IAT Mean'].astype('float32')
df_origin['Flow IAT Std'] = df_origin['Flow IAT Std'].astype('float32')
df_origin['Fwd IAT Mean'] = df_origin['Fwd IAT Mean'].astype('float32')
df_origin['Fwd IAT Std'] = df_origin['Fwd IAT Std'].astype('float32')
df_origin['Bwd IAT Mean'] = df_origin['Bwd IAT Mean'].astype('float32')
df_origin['Bwd IAT Std'] = df_origin['Bwd IAT Std'].astype('float32')
df_origin['Fwd PSH Flags'] = df_origin['Fwd PSH Flags'].astype('int16')
df_origin['Bwd PSH Flags'] = df_origin['Bwd PSH Flags'].astype('int16')
df_origin['Fwd URG Flags'] = df_origin['Fwd URG Flags'].astype('int8')
df_origin['Bwd URG Flags'] = df_origin['Bwd URG Flags'].astype('int8')
df_origin['Fwd Header Length'] = df_origin['Fwd Header Length'].astype('int32')
df_origin['Bwd Header Length'] = df_origin['Bwd Header Length'].astype('int32')
df_origin['Fwd Packets/s'] = df_origin['Fwd Packets/s'].astype('float32')
df_origin['Bwd Packets/s'] = df_origin['Bwd Packets/s'].astype('float32')
df_origin['Packet Length Min'] = df_origin['Packet Length Min'].astype('float32')
df_origin['Packet Length Mean'] = df_origin['Packet Length Mean'].astype('float32')
df_origin['Packet Length Std'] = df_origin['Packet Length Std'].astype('float32')
df_origin['Packet Length Variance'] = df_origin['Packet Length Variance'].astype('float32')
df_origin['FIN Flag Count'] = df_origin['FIN Flag Count'].astype('int8')
df_origin['SYN Flag Count'] = df_origin['SYN Flag Count'].astype('int8')
df_origin['RST Flag Count'] = df_origin['RST Flag Count'].astype('int16')
df_origin['PSH Flag Count'] = df_origin['PSH Flag Count'].astype('int16')
df_origin['ACK Flag Count'] = df_origin['ACK Flag Count'].astype('int32')
df_origin['URG Flag Count'] = df_origin['URG Flag Count'].astype('int8')
df_origin['CWE Flag Count'] = df_origin['CWE Flag Count'].astype('int8')
df_origin['ECE Flag Count'] = df_origin['ECE Flag Count'].astype('int8')
df_origin['Down/Up Ratio'] = df_origin['Down/Up Ratio'].astype('float32')
df_origin['Avg Packet Size'] = df_origin['Avg Packet Size'].astype('float32')
df_origin['Avg Fwd Segment Size'] = df_origin['Avg Fwd Segment Size'].astype('float32')
df_origin['Avg Bwd Segment Size'] = df_origin['Avg Bwd Segment Size'].astype('float32')
df_origin['Fwd Avg Bytes/Bulk'] = df_origin['Fwd Avg Bytes/Bulk'].astype('int32')
df_origin['Fwd Avg Packets/Bulk'] = df_origin['Fwd Avg Packets/Bulk'].astype('int16')
df_origin['Fwd Avg Bulk Rate'] = df_origin['Fwd Avg Bulk Rate'].astype('int32')
df_origin['Bwd Avg Bytes/Bulk'] = df_origin['Bwd Avg Bytes/Bulk'].astype('int32')
df_origin['Bwd Avg Packets/Bulk'] = df_origin['Bwd Avg Packets/Bulk'].astype('int32')
df_origin['Bwd Avg Bulk Rate'] = df_origin['Bwd Avg Bulk Rate'].astype('int32')
df_origin['Subflow Fwd Packets'] = df_origin['Subflow Fwd Packets'].astype('int32')
df_origin['Subflow Fwd Bytes'] = df_origin['Subflow Fwd Bytes'].astype('int32')
df_origin['Subflow Bwd Packets'] = df_origin['Subflow Bwd Packets'].astype('int32')
df_origin['Subflow Bwd Bytes'] = df_origin['Subflow Bwd Bytes'].astype('int32')
df_origin['Init Fwd Win Bytes'] = df_origin['Init Fwd Win Bytes'].astype('int32')
df_origin['Init Bwd Win Bytes'] = df_origin['Init Bwd Win Bytes'].astype('int32')
df_origin['Fwd Act Data Packets'] = df_origin['Fwd Act Data Packets'].astype('int32')
df_origin['Fwd Seg Size Min'] = df_origin['Fwd Seg Size Min'].astype('int8')
df_origin['Active Mean'] = df_origin['Active Mean'].astype('float32')
df_origin['Active Std'] = df_origin['Active Std'].astype('float32')
df_origin['Active Max'] = df_origin['Active Max'].astype('float64')
df_origin['Active Min'] = df_origin['Active Min'].astype('float64')
df_origin['Idle Mean'] = df_origin['Idle Mean'].astype('float32')
df_origin['Idle Std'] = df_origin['Idle Std'].astype('float32')
df_origin['Idle Max'] = df_origin['Idle Max'].astype('float64')
df_origin['Idle Min'] = df_origin['Idle Min'].astype('float64')

In [20]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3630918 entries, 26 to 362173
Data columns (total 79 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Protocol                  int8   
 1   Flow Duration             int64  
 2   Total Fwd Packets         int32  
 3   Total Backward Packets    int32  
 4   Fwd Packets Length Total  float64
 5   Bwd Packets Length Total  float64
 6   Fwd Packet Length Max     float64
 7   Fwd Packet Length Min     float32
 8   Fwd Packet Length Mean    float32
 9   Fwd Packet Length Std     float32
 10  Bwd Packet Length Max     float64
 11  Bwd Packet Length Min     float32
 12  Bwd Packet Length Mean    float32
 13  Bwd Packet Length Std     float32
 14  Flow Bytes/s              float64
 15  Flow Packets/s            float64
 16  Flow IAT Mean             float32
 17  Flow IAT Std              float32
 18  Flow IAT Max              float64
 19  Flow IAT Min              float64
 20  Fwd IAT Total             flo

In [21]:
df_origin.to_parquet(f"./CoAt_CIC-UNSW-NB15_Feeded-V2.parquet")